In [0]:
import dataiku
from langchain.chains.question_answering import load_qa_chain
from dataiku.langchain.dku_llm import DKUChatLLM
import json
KB_IDs = {
    "tech_docs": "zQ92IhQ9",
    "non_conformities": "WnKb6p17"
}
k=10
client = dataiku.api_client()
project = client.get_default_project()
agents = {
    "query": "compute_nc_scenarios_query",
    "nc_search": "compute_nc_scenarios_search_nc",
    "doc_search": "compute_nc_scenarios_search_techdocs",
    "000": "compute_nc_scenarios_propose_000",
    "propose_000": "compute_nc_scenarios_propose_000",
    "propose_100": "compute_nc_scenarios_propose_100",
    "100": "compute_nc_scenarios_propose_100"
}

In [0]:
# Listing available LLMs
llm_list = project.list_llms()

for llm in llm_list:
    print(f"- {llm.description} (id: {llm.id})")

In [0]:
# Fill with your LLM id
LLM_ID = "openai:OpenAI-FA:gpt-4o-mini"

In [0]:
# Preparing the Knowledge Bank, Vector store and LLM
KBs = {
    key: dataiku.KnowledgeBank(id=value, project_key=project.project_key)
    for key, value in KB_IDs.items()
}
vector_stores = {
    key: value.as_langchain_vectorstore()
    for key, value in KBs.items()
}
k = {
    "tech_docs": 40,
    "non_conformities": 20
} # number of docs to retrive


In [0]:
# Create the question answering chain
#chain = load_qa_chain(langchain_llm, chain_type="stuff")
user_message = """
Description du Problème : 

Lors du contrôle de qualité du numéro d’avion MSN 0070, une non-conformité a été identifiée concernant le perçage d'une série de rivets sur le revêtement extérieur, sous la glace du pare-brise droit. Un désaffleurement a été mesuré entre -0,20 mm et -0,25 mm, dépassant les tolérances spécifiées dans les normes d'assemblage. 

Détails Techniques : 

Localisation : Zone en dessous du pare-brise droit 

Mesure de Désaffleurement : -0,20 mm à -0,25 mm 

Norme Acceptable : Tolérance maximale de -0,10 mm selon la spécification interne (Réf. SP-2023-078) 
"""

#user_message = "ma lamborghini est noir et devrait être rouge"


In [0]:
recipe = project.get_recipe('compute_nc_scenarios_query')

# Récupérer la configuration actuelle de la recette
settings = recipe.get_settings()  # Correctement utilisé pour récupérer la configuration
print("Configuration actuelle :", settings.get_json_payload())


In [0]:
def exec_prompt_recipe(recipe_name, inputs):
    #partial method
    recipe = project.get_recipe(recipe_name)
    config = recipe.get_settings().get_json_payload()
    promptStudioId = { "id": config["associatedPromptStudioId"], "prompt_id": config["associatedPromptStudioPromptId"] }
    llm_id = config["llmId"]
    prompt_inputs = config["prompt"]["textPromptTemplateInputs"]
    system_prompt = config["prompt"]["textPromptSystemTemplate"]
    user_prompt = config["prompt"]["textPromptTemplate"]
    temperature = config["completionSettings"]["temperature"]
    for input_def in prompt_inputs:
        placeholder = f"{{{{{input_def['name']}}}}}"  # Exemple : {{description}}
        system_prompt = system_prompt.replace(placeholder, inputs[input_def["name"]])
        user_prompt = user_prompt.replace(placeholder, inputs[input_def["name"]])
    llm = project.get_llm(llm_id)
    completion = llm.new_completion()
    completion.settings["temperature"] = temperature
    completion.with_message(system_prompt, role='system')
    completion.with_message(user_prompt, role='user')
    resp = completion.execute()
    try:
        return json.loads(resp.text)
    except:
        return resp.text


In [0]:
query_inputs = {
    "role": "000",
    "description" : user_message
}
query = exec_prompt_recipe(agents["query"], query_inputs)

print(query)

In [0]:
search_nc = exec_prompt_recipe(agents["nc_search"], {"input": query})
search_docs = exec_prompt_recipe(agents["doc_search"], {"input": query})
print(search_docs["result"])

#search_results = []
#if (query):
#    search_results = [result for key, value in vector_stores.items() for result in value.similarity_search_with_relevance_scores(query, k=10)]
#    #print(search_results)
#   search_results = [ {
#            "doc": doc.metadata['doc'],
#            "chunk_id": doc.metadata['chunk_id'],
#            "relevance_scores": score,
#            "chunk": doc.page_content
#        }
#        for doc, score in search_results
#    ]


search_results = {
    key: value.similarity_search(query)
    for key, value in vector_stores.items()
}

for key in KB_IDs:
    for search_result in search_results[key]:
        print(f"# {search_result.doc} \n{search_result.page_content}\n")

In [0]:
{
    "role": role,
    "description": user_message,
    "search_docs": doc_search,
    "search_nc": search_nc
}

response_content = exec_prompt_recipe(agents[role], )




In [0]:
response_content = json.loads(resp.text)
deep_chat_response = {
    "text": response_content['comment'],
    "label": response_content['label'],
    "description": response_content['description'],
    "sources": search_results,
    "role": "ai"
}
print(deep_chat_response)

In [0]:
auth_info = client.get_auth_info(with_secrets=True)
secret_value = None
print(auth_info["secrets"])
for secret in auth_info["secrets"]:
    if secret["key"] == "JWT_SECRET_KEY":
        secret_value = secret["value"]
        break
print(secret_value)